# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started. Once you have completed the code you can download the notebook for making a submission.

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [3]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 64 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
def generator(source_path, folder_list, batch_size, ablation = False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    #create a list of image numbers you want to use for a particular video
    #Since back-to-back frames will have similar information we can choose only the even frames
    img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
    while True:
        if ablation==True: 
          f = np.random.permutation(folder_list)
          t = np.random.choice(f, 10) # work with only 10 folders for ablation
        else:
          t = np.random.permutation(folder_list)
        # calculate the number of batches
        num_batches = int(len(t)/batch_size)
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                   
                    if image.shape[1] == 160:
                        image = resize(image[:,20:140,:],(120,120)).astype(np.float32)
                    else:
                        image = resize(image,(120,120)).astype(np.float32)


                    #Normalizing using the mean of the 3 channels.
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123


                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,len(img_idx),120,120,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if image.shape[1] == 160:
                        image = resize(image[:,20:140,:],(120,120)).astype(np.float32)
                    else:
                        image = resize(image,(120,120)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model RESNET + CNN
Here you make the model using different functionalities that Keras provides. You might want to use `TimeDistributed`, `GRU` and other RNN structures after doing transfer learning. Also remember that the last layer is the softmax. Remember that the network is designed in such a way that the model is able to fit in the memory of the webcam.

In [24]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Dropout, Flatten, TimeDistributed
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.applications.vgg16 import VGG16

base_model = VGG16(include_top=False, weights='imagenet', input_shape=(120,120,3))
x = base_model.output
x = Flatten()(x)
#x.add(Dropout(0.5))
features = Dense(64, activation='relu')(x)
conv_model = Model(inputs=base_model.input, outputs=features)

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(TimeDistributed(conv_model, input_shape=(18,120,120,3)))
model.add(GRU(32, return_sequences=True))
model.add(GRU(16))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [25]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_2 (TimeDis  (None, 18, 64)           15009664  
 tributed)                                                       
                                                                 
 gru_7 (GRU)                 (None, 18, 32)            9408      
                                                                 
 gru_8 (GRU)                 (None, 16)                2400      
                                                                 
 dropout_4 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 8)                 136       
                                                                 
 dense_8 (Dense)             (None, 5)                 45        
                                                      

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

### Running Abalations and trying to overfit the model. 

In [20]:
!mkdir -p model_output/LSTM

In [26]:

model_name = 'model_output/LSTM/'+'model_conv_lstm' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

In [27]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [28]:
train_generator = generator(train_path, train_doc, batch_size, ablation=True)
val_generator = generator(val_path, val_doc, batch_size, ablation=True)

# Run ablation for 5 EPOCHS
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 64
Epoch 1/5
11/11 [==============================] - ETA: 0s - loss: 1.6654 - categorical_accuracy: 0.2636Source path =  /home/datasets/Project_data/val ; batch size = 64

Epoch 00001: saving model to model_output/LSTM/model_conv_lstm_2023-10-0607_44_34.465621/model-00001-1.66537-0.26364-1.60453-0.15000.h5
11/11 [==============================] - 23s 2s/step - loss: 1.6654 - categorical_accuracy: 0.2636 - val_loss: 1.6045 - val_categorical_accuracy: 0.1500 - lr: 0.0100
Epoch 2/5
11/11 [==============================] - ETA: 0s - loss: 1.6594 - categorical_accuracy: 0.2545
Epoch 00002: saving model to model_output/LSTM/model_conv_lstm_2023-10-0607_44_34.465621/model-00002-1.65940-0.25455-1.51589-0.40000.h5
11/11 [==============================] - 18s 2s/step - loss: 1.6594 - categorical_accuracy: 0.2545 - val_loss: 1.5159 - val_categorical_accuracy: 0.4000 - lr: 0.0100
Epoch 3/5
11/11 [==============================] - ETA

In [31]:
# We can see poor performance in the ablation. 
# Lets try modifing the model


base_model = VGG16(include_top=False, weights='imagenet', input_shape=(120,120,3))
x = base_model.output
x = Flatten()(x)
#x.add(Dropout(0.5))
features = Dense(64, activation='relu')(x)
conv_model = Model(inputs=base_model.input, outputs=features)

for layer in base_model.layers:
    layer.trainable = False

model1 = Sequential()
model1.add(TimeDistributed(conv_model, input_shape=(18,120,120,3)))
model1.add(GRU(64, return_sequences=True))
model1.add(GRU(32, return_sequences=True))
model1.add(Dropout(0.5))
model1.add(GRU(16))
model1.add(Dropout(0.5))
model1.add(Dense(8, activation='relu'))
model1.add(Dense(5, activation='softmax'))


In [32]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.7, nesterov=True)
model1.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_2 (TimeDis  (None, 18, 64)           15009664  
 tributed)                                                       
                                                                 
 gru_7 (GRU)                 (None, 18, 32)            9408      
                                                                 
 gru_8 (GRU)                 (None, 16)                2400      
                                                                 
 dropout_4 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 8)                 136       
                                                                 
 dense_8 (Dense)             (None, 5)                 45        
                                                      

In [34]:
train_generator = generator(train_path, train_doc, batch_size, ablation=True)
val_generator = generator(val_path, val_doc, batch_size, ablation=True)

# Run ablation for 10 EPOCHS
model1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 64
Epoch 1/10
11/11 [==============================] - ETA: 0s - loss: 1.5525 - categorical_accuracy: 0.2818Source path =  /home/datasets/Project_data/val ; batch size = 64

Epoch 00001: saving model to model_output/LSTM/model_conv_lstm_2023-10-0607_44_34.465621/model-00001-1.55255-0.28182-1.65522-0.05000.h5
11/11 [==============================] - 16s 2s/step - loss: 1.5525 - categorical_accuracy: 0.2818 - val_loss: 1.6552 - val_categorical_accuracy: 0.0500 - lr: 0.0100
Epoch 2/10
11/11 [==============================] - ETA: 0s - loss: 1.6138 - categorical_accuracy: 0.2636
Epoch 00002: saving model to model_output/LSTM/model_conv_lstm_2023-10-0607_44_34.465621/model-00002-1.61384-0.26364-1.57794-0.30000.h5
11/11 [==============================] - 15s 1s/step - loss: 1.6138 - categorical_accuracy: 0.2636 - val_loss: 1.5779 - val_categorical_accuracy: 0.3000 - lr: 0.0100
Epoch 3/10
11/11 [==============================] - 

#### Still not overfitting on small data with 10 folders ad 10 epochs lets try resnet

In [44]:
from tensorflow.keras.applications.resnet50 import ResNet50
    
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(120,120,3))
x = base_model.output
x = Flatten()(x)
#x.add(Dropout(0.5))
features = Dense(64, activation='relu')(x)
conv_model = Model(inputs=base_model.input, outputs=features)
    
for layer in base_model.layers:
    layer.trainable = False
        
model2 = Sequential()
model2.add(TimeDistributed(conv_model, input_shape=(18,120,120,3)))
model2.add(GRU(256, return_sequences=True))
model2.add(GRU(128, return_sequences=True))
model2.add(Dropout(0.5))
model2.add(GRU(64, return_sequences=True))
model2.add(GRU(32, return_sequences=True))
model2.add(Dropout(0.5))
model2.add(GRU(16))
model2.add(Dense(8, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(5, activation='softmax'))

In [46]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model2.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_7 (TimeDis  (None, 18, 64)           25684928  
 tributed)                                                       
                                                                 
 gru_25 (GRU)                (None, 18, 256)           247296    
                                                                 
 gru_26 (GRU)                (None, 18, 128)           148224    
                                                                 
 dropout_14 (Dropout)        (None, 18, 128)           0         
                                                                 
 gru_27 (GRU)                (None, 18, 64)            37248     
                                                                 
 gru_28 (GRU)                (None, 18, 32)            9408      
                                                      

In [47]:
train_generator = generator(train_path, train_doc, batch_size, ablation=True)
val_generator = generator(val_path, val_doc, batch_size, ablation=True)

# Run ablation for 10 EPOCHS
model2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 64
Epoch 1/10
11/11 [==============================] - ETA: 0s - loss: 1.5601 - categorical_accuracy: 0.3455Source path =  /home/datasets/Project_data/val ; batch size = 64

Epoch 00001: saving model to model_output/LSTM/model_conv_lstm_2023-10-0607_44_34.465621/model-00001-1.56009-0.34545-1.53069-0.30000.h5
11/11 [==============================] - 27s 2s/step - loss: 1.5601 - categorical_accuracy: 0.3455 - val_loss: 1.5307 - val_categorical_accuracy: 0.3000 - lr: 0.0010
Epoch 2/10
11/11 [==============================] - ETA: 0s - loss: 1.5559 - categorical_accuracy: 0.2909
Epoch 00002: saving model to model_output/LSTM/model_conv_lstm_2023-10-0607_44_34.465621/model-00002-1.55589-0.29091-1.54636-0.45000.h5
11/11 [==============================] - 16s 2s/step - loss: 1.5559 - categorical_accuracy: 0.2909 - val_loss: 1.5464 - val_categorical_accuracy: 0.4500 - lr: 0.0010
Epoch 3/10
11/11 [==============================] - 

In [40]:
# Lets try chnaging the Learning rate

sgd = tf.keras.optimizers.SGD(learning_rate=0.1, decay=1e-2, momentum=0.7, nesterov=True)
model2.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_6 (TimeDis  (None, 18, 64)           25684928  
 tributed)                                                       
                                                                 
 gru_20 (GRU)                (None, 18, 256)           247296    
                                                                 
 gru_21 (GRU)                (None, 18, 128)           148224    
                                                                 
 dropout_11 (Dropout)        (None, 18, 128)           0         
                                                                 
 gru_22 (GRU)                (None, 18, 64)            37248     
                                                                 
 gru_23 (GRU)                (None, 18, 32)            9408      
                                                      

#### As we can see the accuracy is growing slowing, so if we run this for higher epochs it will over fit

#### Lets Reduce the params of the Model and run ablations again

In [11]:
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(120,120,3))
x = base_model.output
x = Flatten()(x)
#x.add(Dropout(0.5))
features = Dense(64, activation='relu')(x)
conv_model = Model(inputs=base_model.input, outputs=features)
    
for layer in base_model.layers:
    layer.trainable = False
        
model_res_cnn = Sequential()
model_res_cnn.add(TimeDistributed(conv_model, input_shape=(15,120,120,3)))
model_res_cnn.add(GRU(64, return_sequences=True))
model_res_cnn.add(Dropout(0.5))
model_res_cnn.add(GRU(32, return_sequences=True))
model_res_cnn.add(GRU(16))
model_res_cnn.add(Dropout(0.5))
model_res_cnn.add(Dense(8, activation='relu'))
model_res_cnn.add(Dense(5, activation='softmax'))

In [12]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.7, nesterov=True)
model_res_cnn.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_res_cnn.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_1 (TimeDis  (None, 15, 64)           25684928  
 tributed)                                                       
                                                                 
 gru_5 (GRU)                 (None, 15, 64)            24960     
                                                                 
 dropout_3 (Dropout)         (None, 15, 64)            0         
                                                                 
 gru_6 (GRU)                 (None, 15, 32)            9408      
                                                                 
 gru_7 (GRU)                 (None, 16)                2400      
                                                                 
 dropout_4 (Dropout)         (None, 16)                0         
                                                      

In [13]:
train_generator = generator(train_path, train_doc, batch_size, ablation=True)
val_generator = generator(val_path, val_doc, batch_size, ablation=True)

# Ru ablation for 5 EPOCHS
model_res_cnn.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_302/3015679548.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_res_cnn.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=5, verbose=1,


Source path =  /home/datasets/Project_data/train ; batch size = 50
Epoch 1/5
14/14 [==============================] - ETA: 0s - loss: 1.6833 - categorical_accuracy: 0.1571Source path =  /home/datasets/Project_data/val ; batch size = 50

Epoch 00001: saving model to model_init_conv_lstm_2023-10-0213_11_18.391040/model-00001-1.68325-0.15714-1.59328-0.25000.h5
14/14 [==============================] - 19s 951ms/step - loss: 1.6833 - categorical_accuracy: 0.1571 - val_loss: 1.5933 - val_categorical_accuracy: 0.2500 - lr: 0.0100
Epoch 2/5
14/14 [==============================] - ETA: 0s - loss: 1.6441 - categorical_accuracy: 0.1500
Epoch 00002: saving model to model_init_conv_lstm_2023-10-0213_11_18.391040/model-00002-1.64406-0.15000-1.60181-0.30000.h5
14/14 [==============================] - 13s 991ms/step - loss: 1.6441 - categorical_accuracy: 0.1500 - val_loss: 1.6018 - val_categorical_accuracy: 0.3000 - lr: 0.0100
Epoch 3/5
14/14 [==============================] - ETA: 0s - loss: 1.6262 

### Lets Run it with full data and 100 epochs

In [14]:
train_generator = generator(train_path, train_doc, batch_size, ablation=False)
val_generator = generator(val_path, val_doc, batch_size, ablation=False)


model_res_cnn.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_302/1818576770.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_res_cnn.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /home/datasets/Project_data/train ; batch size = 50
Epoch 1/100
14/14 [==============================] - ETA: 0s - loss: 1.6133 - categorical_accuracy: 0.2413Source path =  /home/datasets/Project_data/val ; batch size = 50

Epoch 00001: saving model to model_init_conv_lstm_2023-10-0213_11_18.391040/model-00001-1.61333-0.24133-1.60666-0.26000.h5
14/14 [==============================] - 59s 4s/step - loss: 1.6133 - categorical_accuracy: 0.2413 - val_loss: 1.6067 - val_categorical_accuracy: 0.2600 - lr: 0.0025
Epoch 2/100
14/14 [==============================] - ETA: 0s - loss: 1.5956 - categorical_accuracy: 0.2443
Epoch 00002: saving model to model_init_conv_lstm_2023-10-0213_11_18.391040/model-00002-1.59556-0.24434-1.60605-0.28000.h5
14/14 [==============================] - 59s 5s/step - loss: 1.5956 - categorical_accuracy: 0.2443 - val_loss: 1.6060 - val_categorical_accuracy: 0.2800 - lr: 0.0025
Epoch 3/100
14/14 [==============================] - ETA: 0s - loss: 1.5882 

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(16,120,120,3)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

# model.add(Dropout(0.25))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

# model.add(Dropout(0.25))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [15]:
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_24 (Conv3D)          (None, 16, 120, 120, 64)  5248      
                                                                 
 batch_normalization_24 (Bat  (None, 16, 120, 120, 64)  256      
 chNormalization)                                                
                                                                 
 activation_24 (Activation)  (None, 16, 120, 120, 64)  0         
                                                                 
 max_pooling3d_24 (MaxPoolin  (None, 8, 60, 120, 64)   0         
 g3D)                                                            
                                                                 
 conv3d_25 (Conv3D)          (None, 8, 60, 120, 128)   221312    
                                                                 
 batch_normalization_25 (Bat  (None, 8, 60, 120, 128) 

/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


#### Run Ablations

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [19]:
train_generator = generator(train_path, train_doc, 64, ablation=False)
val_generator = generator(val_path, val_doc, 64, ablation=False)

In [20]:
model_name = '3D_convo' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = num_epochs)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [21]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

#### Run for 10 epochs

In [22]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=10, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 64
Epoch 1/10


2023-10-03 18:00:33.379947: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302
2023-10-03 18:00:37.072700: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: required broadcastable shapes


InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[0,5] labels_size=[64,5]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits
 (defined at /usr/local/lib/python3.8/dist-packages/keras/backend.py:5009)
]] [Op:__inference_train_function_3124]

Errors may have originated from an input operation.
Input Source operations connected to node categorical_crossentropy/softmax_cross_entropy_with_logits:
In[0] categorical_crossentropy/softmax_cross_entropy_with_logits/Reshape:	
In[1] categorical_crossentropy/softmax_cross_entropy_with_logits/Reshape_1:

Operation defined at: (most recent call last)
>>>   File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 16, in <module>
>>>     app.launch_new_instance()
>